In [37]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.visualization import plot_histogram
from qiskit import Aer, execute
from qiskit.circuit import Parameter
import networkx as nx
from copy import deepcopy
import numpy as np
from itertools import permutations


In [38]:
#### HELPER FUNCTION ####
def string_to_arr(string):
    arr = []
    for str in string: arr.append(int(str))
    return np.array(arr)
    
def score(n1,n2):
    """ Function for computing the score of
        some alignment of the two chars n1, n2
    Args:
        n1: any str in {"_","A","T","C","G"}
        n2: any str in {"_","A","T","C","G"}
    Returns:
        score: int in {-1,0,1}
    """
    score = None
    gap = "_"
    if   n1 == gap or n2 == gap: score =  0; return score
    elif n1 == n2              : score = -1; return score
    elif n1 != n2              : score =  1; return score

def score_sequence(arr):
    """Function for computing sum-of-pairs score
       according to scheme defined in 
       score function. 
    Args:
        arr: numpy array, e.g.: np.array(["A","C","_","T","_"])
    Returns:
        final_score: integer 
       """
    final_score = 0
    for n1 in range(0 , len(arr)):
        for n2 in range(n1 + 1 , len(arr)):
            final_score += score(arr[n1],arr[n2])
    return final_score

def recursive_perm_scoring(mat, perms, test_mat, row_idx):
    """Computing all possibles alignment scores of MSA matrix; mat,
       given the permutations for each of the N-1 rows of mat, (found in perms)
    
    Args:
        mat     : 2D numpy array, e.g. array([['A', 'C', 'C', 'T'],
                                              ['A', 'C', '_', '_'],
                                              ['A', 'T', '_', '_']])
        perms   : all distinct permutations of the N-1 last rows of mat
        test_mat: matrix 
       """
    perm_history, score_history = [], []
    if row_idx < mat.shape[0]:
        for i in range(0 , len(perms[row_idx - 1])):
            test_mat[row_idx,:] = perms[row_idx - 1][i]
            score_list, perm_list = recursive_perm_scoring(mat, perms, test_mat, row_idx + 1)
            perm_history += perm_list
            score_history += score_list
            current_score = 0
            for j in range(0 , mat.shape[1]):
                current_score += score_sequence(test_mat[:,j])
            score_history.append(current_score)
            perm_history.append(deepcopy(test_mat))
    return score_history, perm_history

def matrix_2_bit_state(mat):
    """
    Maps some given matrix repr. of a MSA to a corresponding
    bitstring via the column encoding: x_(s,n,i) determines whether
    the n'th letter of the s'th string is placed in the i'th column.

    Args:
        mat: 2D numpy array, e.g. array([['A', 'C', 'C', 'T'],
                                         ['A', 'C', '_', '_'],
                                         ['A', 'T', '_', '_']])
    Returns:
        numpy array containing bit repr., e.g.: np.array([1,0,0,1...])
    """
    regs = []
    gap = "_"
    for row in range(mat.shape[0]):           # For each S
        for col in range(mat.shape[1]):       # For each n
            if mat[row][col] != gap:
                current_reg = [] 
                for i in range(mat.shape[1]):     # for each i
                    if i == col: current_reg.append(1)
                    else:        current_reg.append(0)
                regs.append(current_reg)
    return np.array(regs).flatten()

def bit_state_2_matrix(bit_string,init_mat):
    """
    Maps some given bitstring repr. of a MSA to a corresponding
    matrix via the initial matrix. 

    Args:
        bit_string: numpy array containing bit repr., e.g.: np.array([1,0,0,1...])
        mat       : 2D numpy array, e.g. array([['A', 'C', 'C', 'T'],
                                                ['A', 'C', '_', '_'],
                                                ['A', 'T', '_', '_']])
    Returns:
        2D numpy array, e.g. array([['A', 'C', 'C', 'T'],
                                    ['A', 'C', '_', '_'],
                                    ['A', 'T', '_', '_']])
    """
    counts, letters, gap = [], [], "_"
    for row in range(init_mat.shape[0]):
        current_count = 0
        current_letters  = []
        for col in range(init_mat.shape[1]):
            if init_mat[row][col] != gap: 
                current_count += 1
                current_letters.append(init_mat[row][col])
        letters.append(current_letters)
        counts.append(current_count)

    lower = 0
    multiplier, regs = init_mat.shape[1], []
    for value in counts:
        for i in range(value):
            regs.append(bit_string[lower + i * multiplier : lower + (i + 1) * multiplier])
        lower += value * multiplier

    counter = 0
    new_mat = np.zeros((init_mat.shape),dtype=object)
    counter = 0
    for i in range(len(letters)):
        for j in range(len(letters[i])):
            col_idx = np.where(regs[counter] == 1)[0][0]
            new_mat[i][col_idx] = letters[i][j]
            counter += 1

    new_mat[new_mat == 0] = "_"
    return new_mat

def initial_MSA_matrix(strings):
    """ Creating a matrix representation of the strings given
    and filling gaps with "_"

    Args:
        list of strings, e.g. ["ACCT","AC","AT"]

    Returns:
        2D numpy array
    """
    lengths = [len(str) for str in strings]
    initial_matrix = np.zeros((len(strings) , np.max(lengths)),dtype=object)
    for row in range(initial_matrix.shape[0]):
        for col in range(len(strings[row])):
            initial_matrix[row][col] = strings[row][col]
    initial_matrix[initial_matrix == 0] = "_"
    return initial_matrix

def legal_permutations(arr): 
    """ Function for computing all permutations of array
    of type ["A","C","_","T","_"] that maintains original
    order of characters != "_" .

    Args:
        arr: numpy array, e.g.: np.array(["A","C","_","T","_"])
    Returns:
        2D numpy array of legal permutations, e.g.: np.array([["A","C","_","T","_"],
                                                              ["A","C","_","_","T"],...])

    """
    legal_perm_indices = []
    letter_order = [char for char in arr if char != "_"]
    perms = list(set(permutations(arr))) # Using set to remove dubs
    perms = [list(perm) for perm in perms]
    for idx, perm in enumerate(perms):
        letter_counter = 0
        keep_perm = True
        for letter in perm:
            if letter != "_":
                if letter_order[letter_counter] != letter:
                    keep_perm = False
                else: letter_counter += 1
        if keep_perm: legal_perm_indices.append(idx)
    return np.array(perms)[legal_perm_indices]


def score_sequence(arr):
    """Function for computing sum-of-pairs score
       according to scheme defined in 
       score function. 
    Args:
        arr: numpy array, e.g.: np.array(["A","C","_","T","_"])
    Returns:
        final_score: integer 
       """
    final_score = 0
    for n1 in range(0 , len(arr)):
        for n2 in range(n1 + 1 , len(arr)):
            final_score += score(arr[n1],arr[n2])
    return final_score

def recursive_brute_force(init_mat):
    """ Final recursive version of brute force scoring of all relevant
    permutations of variable sized MSA matrix. 
    (Make sure the longest row containin only letters are
     initially placed at top of matrix)
     
    Args:
        Init_mat: 2D numpy array, e.g. array([['A', 'C', 'C', 'T'],
                                              ['A', 'C', '_', '_'],
                                              ['A', 'T', '_', '_']])
    Returns:
        best_score: Integer
        best perms: list of corresponding permutations of MSA matrix
    """
    perms = []
    for row_idx in range(1 , init_mat.shape[0]):
        perms.append(legal_permutations(init_mat[row_idx,:]))
    test_mat = np.zeros((init_mat.shape) , dtype=object)
    test_mat[0,:] = init_mat[0,:]
    score_history, mat_history = recursive_perm_scoring(init_mat,perms,test_mat,1)
    best_score = np.min(score_history)
    best_perms = [mat_history[i] for i in range(len(mat_history)) if score_history[i] == best_score]
    return best_score, best_perms


def encode_score_weights(mat):
    """Encoding the score of all possible alignments
        for all n1, n2 for all s1 < s2 score(n1,n2)

    Args:
        mat: 2D numpy array, e.g. array([['A', 'C', 'C', 'T'],
                                         ['A', 'C', '_', '_'],
                                         ['A', 'T', '_', '_']])   
    Returns:
        weight_matrices: 3D numpy array of shape (1/2 * (L - 1) * L , C , C)
                         where L = nr. of rows and C = nr. of cols in
                         matrix given
    """
    L, C = mat.shape
    weight_matrices = [np.zeros((C,C)) for i in range(int(1/2 * (L - 1) * L))]
    for row1 in range(0 , mat.shape[0]):
        for row2 in range(row1 + 1 , mat.shape[0]):
            for idx1, n1 in enumerate(mat[row1,:]):
                for idx2, n2 in enumerate(mat[row2,:]):
                    weight_matrices[row1+row2-1][idx1][idx2] = score(n1,n2)
    return np.array(weight_matrices)

def score_matrix(mat: np.array) -> int:
    """Function for calculating the alignment score
    of a MSA matrix"""
    final_score = 0
    for col in range(0 , mat.shape[1]):
        final_score += score_sequence(mat[:,col])
    return final_score

In [39]:
DNA_sequences   = ["AC","C"]
Init_DNA_matrix = initial_MSA_matrix(DNA_sequences)
Init_DNA_matrix

array([['A', 'C'],
       ['C', '_']], dtype=object)

In [40]:
Initial_score = score_matrix(Init_DNA_matrix)
Initial_score

1

In [41]:
best_score, perms = recursive_brute_force(Init_DNA_matrix)
best_score, perms


(-1,
 [array([['A', 'C'],
         ['_', 'C']], dtype=object)])

In [42]:
bitstr = matrix_2_bit_state(Init_DNA_matrix)
bitstr

array([1, 0, 0, 1, 1, 0])

In [43]:
bit_state_2_matrix(bitstr,Init_DNA_matrix)

array([['A', 'C'],
       ['C', '_']], dtype=object)

In [44]:
def compute_expectation(counts, initial_MSA):
    
    """
    Computes expectation value based on measurement results
    
    Args:
        counts: dict
                key as bitstring, val as count
           
        initial_MSA: initial 2D np array repr. of MSA
        
    Returns:
        avg: float
             expectation value
    """
    avg = 0
    sum_count = 0
    for bitstring, count in counts.items():
        MSA_mat = bit_state_2_matrix(string_to_arr(bitstring),initial_MSA)
        score   = score_matrix(MSA_mat)
        avg += score * count
        sum_count += count
    return avg/sum_count

In [45]:
def create_qaoa_circ(initial_MSA, theta):
    
    """
    Creates a parametrized qaoa circuit
    
    Args:  
        G: networkx graph
        theta: list
               unitary parameters
                     
    Returns:
        qc: qiskit circuit
    """
    letters_in_mat = [initial_MSA[i][j] for i in range(initial_MSA.shape[0]) 
                      for j in range(initial_MSA.shape[1]) if initial_MSA[i][j] != "_"]
    
    nqubits = int(len(letters_in_mat) *  initial_MSA.shape[1])  # Number of qubits = number of nodes in graph 
    p = len(theta)//2               # Number of alternating unitaries
    qc = QuantumCircuit(nqubits)    # Initializing Q circuit w. nqubits nr. of qbits
    
    beta = theta[:p]                # Beta opt param for mixing unitaries as first p vals.
    gamma = theta[p:]               # Gama opt param for cost unitaries as last p vals.
    
    # Initial_state: Hadamark gate on each qbit
    for i in range(0, nqubits):
        qc.h(i)
    
    # Cost unitary: Z_i*Z_j on pair[0], pair[1] qbits
    for irep in range(0, p):        
        for pair in list(G.edges()):
            qc.rzz(2 * gamma[irep], pair[0], pair[1])

    # mixer unitary: X rotation on each qbit      
    for irep in range(0, p): 
        for i in range(0, nqubits):
            qc.rx(2 * beta[irep], i)
            
    qc.measure_all()        
    return qc


In [46]:
# Finally we write a function that executes the circuit on the chosen backend
def get_expectation(G, p, shots=512):
    
    """
    Runs parametrized circuit
    
    Args:
        G: networkx graph
        p: int,
           Number of repetitions of unitaries
    """
    print(G.nodes,p)
    backend = Aer.get_backend('qasm_simulator')
    backend.shots = shots
    
    def execute_circ(theta):
        
        qc = create_qaoa_circ(G, theta)
        counts = backend.run(qc, seed_simulator=10, 
                             nshots=512).result().get_counts()
        print(counts)
        return compute_expectation(counts, G) ## Returns the expectation of graph, given counts
    
    return execute_circ

In [47]:
from scipy.optimize import minimize

expectation = get_expectation(G, p=2)

res = minimize(fun = expectation, x0 = [1.0,1.0], method='COBYLA')
res.x

NameError: name 'G' is not defined

In [ ]:
backend = Aer.get_backend('aer_simulator')
backend.shots = 512

qc_res = create_qaoa_circ(G, res.x)

counts = backend.run(qc_res, seed_simulator=10).result().get_counts()

plot_histogram(counts,figsize=(20,7))

In [48]:
backend = Aer.get_backend('qasm_simulator')
backend.run()

<bound method AerBackend.properties of QasmSimulator('qasm_simulator')>